[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juansensio/blog/blob/master/076_pbdl_jax_cfd_intro/076_pbdl_jax_cfd_intro.ipynb)

# PBDL con JAX para CFD

Este es el primero en una serie de posts con un triple objetivo de aprendizaje, lo que significa aprender tres cosas a la vez 🤯 (el tiempo es limitado y hay que optimizar). Según el orden en el que aparecen en el título:

- [**PBDL**](https://physicsbaseddeeplearning.org/intro.html): *Physics-Based Deep Learning*, o el uso del *Deep Learning* (redes neuronales) para simulación física. 
- [**JAX**](https://github.com/google/jax): Una librería para computación numérica, con especial énfasis en Inteligencia Artificial.
- [**CFD**](https://es.wikipedia.org/wiki/Mec%C3%A1nica_de_fluidos_computacional): *Computational Fluid Dynamics*, el campo de la física que se enfoca en la simulación de fluidos para aplicaciones de aerodinámica, combustión, etc.

Si te interesa aprender sobre cualquiera de estos tres temas (los cuales por si solos merecen de gran atención), estas en el lugar adecuado 🙃 Sin embargo, te advierto que nos vamos a alejar del *machine learning* tradicional para explorar un nuevo campo, el del uso de las redes neuronales para aproximar soluciones a ecuaciones diferenciales. Es posible que en algunos momentos te preguntes: ¿es esto realmente *machine learning*? Te entiendo. Aún así, creo firmemente que el campo del *PBDL* revolucionará la manera en la que simulamos la naturaleza en los próximos años, de la misma manera que el *Deep Learning* ha revolucionado (y lo sigue haciendo) tantos otros campos de la ciencia, como por ejemplo el [plegado de proteinas](https://deepmind.com/blog/article/AlphaFold-Using-AI-for-scientific-discovery).

## *Physics-based Deep Learning*

El campo del *PBDL* es una disciplina relativamente nueva e inexplorada que se basa el uso de redes neuronales para sustituir (o complementar) métodos numéricos "tradicionales" utilizados desde hace años para simular los diferentes procesos físicos que rigen nuestra naturaleza (desde el comportamiento de nuestra atmósfera hasta el movimiento de estrellas y galaxias). Estos procesos pueden ser descritos, en la mayoría de ocasiones, mediante ecuaciones matemáticas. Resolver estas ecuaciones nos permite calcular, por ejemplo, la distribución de presión sobre una superficie aerodinámica (lo cual es muy útil a la hora de diseñar aviones más eficientes, entre muchas otras aplicaciones). Sin embargo, como te podrás imaginar, estas ecuaciones suelen ser muy difíciles de resolver y, en la mayoría de situaciones, ni siquiera pueden ser resueltas de manera analítica. Es aquí donde entran en juego los métodos numéricos, técnicas que nos permiten aproximar soluciones a estas ecuaciones que si bien no son exactas son lo suficientemente precisas para su uso en aplicaciones reales. Tradicionalmente, métodos numéricos de este estilo requieren de grandes recursos computacionales (es por este motivo que tenemos "superordenadores"). Por lo que cualquier avance en el campo que nos permita encontrar soluciones más rápidas y baratas supone una revolución. Creo que el campo del *PBDL* será la siguiente revolución en este campo. De hecho, este fue el motivo por el que me adentré en el mundo del *Deep Learning*, persiguiendo la idea de que usar [redes neuronales para aproximar soluciones a ecuaciones diferenciales](https://arxiv.org/abs/1912.04737) podía ser una buena idea.

> Recientemente se ha publicado este [libro](https://physicsbaseddeeplearning.org/intro.html) sobre *PBDL*. No dudes en consultarlo para aprender más !

## *Computational Fluid Dynamics*

Dentro del gran abanico de aplicaciones de la física computacional, la mecánica de fluidos computacional se encarga del estudio del comportamiento de fluidos, principalmente mediante la resolución de las ecuaciones de [Navier-Stokes](https://es.wikipedia.org/wiki/Ecuaciones_de_Navier-Stokes). Esto tiene un uso muy importante en el diseño de aeronaves, coches (muy importante en coches eléctricos), previsión meteorológica y análisis de la evolución de contaminantes, etc. 

![](https://upload.wikimedia.org/wikipedia/commons/thumb/5/55/X-43A_%28Hyper_-_X%29_Mach_7_computational_fluid_dynamic_%28CFD%29.jpg/1024px-X-43A_%28Hyper_-_X%29_Mach_7_computational_fluid_dynamic_%28CFD%29.jpg)

Como ya he comentado anteriormente, resolver estas ecuaciones de manera analítica es imposible y su resolución numérica require de grandes recursos computacionales. Aún así, cada vez es más extendido su uso. En el caso del sector aeronáutico la alternativa es el uso de túneles de viento, lo cual es todavía más caro y lento. Poder diseñar vehículos con software de diseño 3d por ordenador, simular su comportamiento en varias condiciones e iterar su diseño hasta encontrar la geometría óptima en entornos virtuales es una gran ventaja. Creo que el uso del *Deep Learning* para *CFD* supondrá una revolución y acelerará, a la vez que abaratará, todo este proceso dando como resultado vehículos más eficientes, que viajen más rápido consumiendo y contaminando menos. 

> Si quieres aprender más sobre *CFD* te recomiendo echarle un vistazo a mi [tesis doctoral](https://www.tesisenred.net/handle/10803/667041#page=1) 🤗

## JAX

Para explorar el mundo del *PBDL* para *CFD* usaremos la librería [JAX](https://github.com/google/jax). Desarrollada y mantenida por Google, JAX es una librería para cálculo numérico en Python con un enfoque particular en *machine learning*, ya que nos va a permitir calcular derivadas de forma automática y ejecutar nuestras operaciones en GPUs y TPUs de manera sencilla. Similar en espíritu a Tensorflow y Pytorch, la principal diferencia de JAX es su API minimalista y sencilla.

### Instalación

Para instalar JAX puedes seguir la [instrucciones](https://github.com/google/jax#installation) en Github. La versión en CPU la puedes instalar en Ubuntu, MacOS y Windows. La versión GPU solo la podrás instalar en Ubuntu, donde necesitarás tener instalado CUDA y CUDNN. En Google Colab, ya lo tendrás instalado y listo para ser usado tanto en CPU, GPU y TPU. Una vez instalado, puedes probar que todo está bien.

In [1]:
import jax

jax.__version__

'0.2.21'

### Conceptos básicos

Lo primero que tienes que saber acerca de JAX es que es muy similar a NumPy, por lo que lo podrás usar para lo mismo (más algunos beneficios que veremos más adelante). Uno de los usos principales de NumPy, que usaremos mucho a la hora de simular fluidos y que también se usa a la hora de entrenar redes neuronales, es la multiplicación de matrices. 

In [97]:
import jax.numpy as jnp

# esto es un vector

x = jnp.arange(3)
x

DeviceArray([0, 1, 2], dtype=int32)

In [14]:
# esto es una matriz

I = jnp.eye(3)
I

DeviceArray([[1., 0., 0.],
             [0., 1., 0.],
             [0., 0., 1.]], dtype=float32)

In [15]:
# multiplicación matriz-vector 

I @ x

DeviceArray([0., 1., 2.], dtype=float32)

### Transformaciones

El concepto de `transformaciones` es lo que le da a JAX su flexibilidad y potencia. Estos son algunos ejemplos:

In [73]:
def func(size=1000):
  x = jnp.arange(size)
  I = jnp.eye(size)
  return I @ x

In [77]:
%timeit func()

993 µs ± 28.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Con `jit` aceleraremos nuestros cálculos gracias al *just-in-time compiler*.

In [75]:
from jax import jit

func_jit = jit(func)

In [78]:
%timeit func_jit()

46.5 µs ± 374 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Con `grad` podremos calcular derivadas.

In [79]:
def func2(x):
  return x**2.


In [80]:
x = jnp.arange(3.)

func2(x)

DeviceArray([0., 1., 4.], dtype=float32)

In [81]:
from jax import grad

derivative_fn = grad(func2)

derivative_fn(10.)

DeviceArray(20., dtype=float32)

Con `vmap` podremos auto-vectorizar nuestro código, sin tener que preocuparnos por pensar como pasar nuestros cálculos a modo "batch".

In [90]:
def func3(x):
  I = jnp.eye(x.size)
  return I @ x


In [91]:
x = jnp.arange(3)

func3(x)

DeviceArray([0., 1., 2.], dtype=float32)

In [93]:
x_batch = jnp.stack(3*[x])

x_batch

DeviceArray([[0, 1, 2],
             [0, 1, 2],
             [0, 1, 2]], dtype=int32)

In [96]:
func3(x_batch) # esto no va porque las dimensiones no encajan

TypeError: dot_general requires contracting dimensions to have the same shape, got [9] and [3].

In [ ]:
from jax import vmap 

func_vmap = vmap(func3)

func_vmap(x_batch)

DeviceArray([[0., 1., 2.],
             [0., 1., 2.],
             [0., 1., 2.]], dtype=float32)

Existen otras transformaciones que pueden ser útiles, además las podemos combinar de manera arbitraria. Todo esto lo iremos aprendiendo en posts futuros.